In [ ]:
import os
from netCDF4 import Dataset as NetCDFFile
import time
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from matplotlib import colors, ticker
import numpy.ma as ma
from scipy.interpolate import griddata
#from pyresample import geometry,image, kd_tree
import copy
import isamcalc_lib as isam
import C14utils

In [ ]:
# 1. Revise 14C for case 5
fc14 = 'bgc.isam_initial.case4.nc'
fbgc = 'bgc.isam_initial.done.nc'

In [ ]:
# 2. Use the calculated D14C for calculating back the soc14 based on soc of the new initial file
nclu_obs=NetCDFFile(fc14,'r')
agcbox = nclu_obs.variables['agcbox'][:]
bgcbox = nclu_obs.variables['bgcbox'][:]
agc14box = nclu_obs.variables['agc14box'][:]
bgc14box = nclu_obs.variables['bgc14box'][:]
latnc = nclu_obs.variables['lat'][:]
lonnc = nclu_obs.variables['lon'][:]
nclu_obs.close()

# Calculate the D14C of the old results 
soc_as_c14_agc = np.zeros((24, 4, 360, 720))
soc_del_c14_agc = np.zeros((24, 4, 360, 720))
soc_as_c14_bgc = np.zeros((24, 5, 360, 720))
soc_del_c14_bgc = np.zeros((24, 5, 360, 720))
for j in np.arange(0,24):
    for k in np.arange(0,4):
        soc_as_c14_agc[j,k,:,:] = agc14box[j,k,:,:] / (agcbox[j,k,:,:]+1e-14)
        #soc_del_c14_agc[i,j,k,:,:] = (soc_as_c14_agc[i,j,k,:,:] - 1.) * 1000.

for j in np.arange(0,24):
    for k in np.arange(0,5):
        soc_as_c14_bgc[j,k,:,:] = bgc14box[j,k,:,:] / (bgcbox[j,k,:,:]+1e-14)
        #soc_del_c14_bgc[i,j,k,:,:] = (soc_as_c14_bgc[i,j,k,:,:] - 1.) * 1000.

# Force unreasonable values into 1 if occurs
soc_as_c14_agc[agcbox<1e-4] = 1.
soc_as_c14_bgc[bgcbox<1e-4] = 1.

for j in np.arange(0,24):
    for k in np.arange(0,4):
        #soc_as_c14_agc[j,k,:,:] = agc14[i,j,k,:,:] / (agc[i,j,k,:,:]+1e-14)
        soc_del_c14_agc[j,k,:,:] = (soc_as_c14_agc[j,k,:,:]*1e-12 - 1.) * 1000.

for j in np.arange(0,24):
    for k in np.arange(0,5):
        #soc_as_c14_bgc[j,k,:,:] = bgc14[i,j,k,:,:] / (bgc[i,j,k,:,:]+1e-14)
        soc_del_c14_bgc[j,k,:,:] = (soc_as_c14_bgc[j,k,:,:]*1e-12 - 1.) * 1000.

# # Write into Netcdf to diagnose/check the values
# nc = NetCDFFile('d14c_old_2.nc','w',format='NETCDF4_CLASSIC')
# # Define a set of dimensions to be used in the file
# level = nc.createDimension('level', 10)
# biome = nc.createDimension('biome', 24)
# pool = nc.createDimension('pool', 5)
# lat = nc.createDimension('lat', 360)
# lon = nc.createDimension('lon', 720)
# # Create variables
# levels = nc.createVariable('level', np.int32, ('level',))
# latitudes = nc.createVariable('lat', np.float32, ('lat',))
# longitudes = nc.createVariable('lon', np.float32, ('lon',))
# # Create the actual 4-d variable
# d14c = nc.createVariable('d14c', np.float32, ('level','biome','pool','lat','lon'))

# # Add Global Attributes
# nc.description = 'D14C calculated from the 1D-SBGC restart file.'
# nc.history = 'Created ' + time.ctime(time.time())
# nc.source = 'netCDF4 python module'

# # Add Variable Attributes
# latitudes.units = 'degree_north'
# longitudes.units = 'degree_east'
# d14c.units = 'o/oo'

# # Put longitude and latitude
# lats = np.arange(-89.75,90.25,0.5)
# lons = np.arange(0.25,360.25,0.5)
# levels = np.arange(1,11)
# latitudes[:] = lats
# longitudes[:] = lons
# d14c[:,:,:,:,:] = soc_del_c14_bgc
# nc.close()

In [ ]:
# 3. Use the old D14C to update the new initial file
nclu_obs=NetCDFFile(fbgc,'r')
agcbox = nclu_obs.variables['agcbox'][:]
bgcbox = nclu_obs.variables['bgcbox'][:]
agc14box = nclu_obs.variables['agc14box'][:]
bgc14box = nclu_obs.variables['bgc14box'][:]
latnc = nclu_obs.variables['lat'][:]
lonnc = nclu_obs.variables['lon'][:]
nclu_obs.close()

agc14_new = np.zeros((24, 4, 360, 720))
#soc_del_c14_agc_new = np.zeros((24, 4, 360, 720))
bgc14_new = np.zeros((24, 5, 360, 720))
#soc_del_c14_bgc_new = np.zeros((24, 5, 360, 720))

for j in np.arange(0,24):
    for k in np.arange(0,4):
        agc14_new[j,k,:,:] = soc_as_c14_agc[j,k,:,:]*agcbox[j,k,:,:]
            
for j in np.arange(0,24):
    for k in np.arange(0,5):
        bgc14_new[j,k,:,:] = soc_as_c14_bgc[j,k,:,:]*bgcbox[j,k,:,:]
        
# # Validate after the calculation
# for i in np.arange(0,10):
#     for j in np.arange(0,24):
#         for k in np.arange(0,4):
#             soc_as_c14_agc_new[i,j,k,:,:] = agc14_new[i,j,k,:,:] / (agc_new[i,j,k,:,:]+1e-14)
#             soc_del_c14_agc_new[i,j,k,:,:] = (soc_as_c14_agc_new[i,j,k,:,:] - 1.) * 1000.

# for i in np.arange(0,10):
#     for j in np.arange(0,24):
#         for k in np.arange(0,5):
#             soc_as_c14_bgc_new[i,j,k,:,:] = bgc14_new[i,j,k,:,:] / (bgc_new[i,j,k,:,:]+1e-14)
#             soc_del_c14_bgc_new[i,j,k,:,:] = (soc_as_c14_bgc_new[i,j,k,:,:] - 1.) * 1000.
            
# # Write into Netcdf to check the values
# nc = NetCDFFile('d14c_new_2.nc','w',format='NETCDF4_CLASSIC')
# # Define a set of dimensions to be used in the file
# level = nc.createDimension('level', 10)
# biome = nc.createDimension('biome', 24)
# pool = nc.createDimension('pool', 5)
# lat = nc.createDimension('lat', 360)
# lon = nc.createDimension('lon', 720)
# # Create variables
# levels = nc.createVariable('level', np.int32, ('level',))
# latitudes = nc.createVariable('lat', np.float32, ('lat',))
# longitudes = nc.createVariable('lon', np.float32, ('lon',))
# # Create the actual 4-d variable
# d14c = nc.createVariable('d14c', np.float32, ('level','biome','pool','lat','lon'))

# # Add Global Attributes
# nc.description = 'D14C calculated from the 1D-SBGC restart file.'
# nc.history = 'Created ' + time.ctime(time.time())
# nc.source = 'netCDF4 python module'

# # Add Variable Attributes
# latitudes.units = 'degree_north'
# longitudes.units = 'degree_east'
# d14c.units = 'o/oo'

# # Put longitude and latitude
# lats = np.arange(-89.75,90.25,0.5)
# lons = np.arange(0.25,360.25,0.5)
# levels = np.arange(1,11)
# latitudes[:] = lats
# longitudes[:] = lons
# d14c[:,:,:,:,:] = soc_del_c14_bgc_new
# nc.close()
            

In [ ]:
# 6. Store back the SOC14 into the restart file
fout = 'bgc.isam_initial.adjusted_c14.nc'
nclu_obs=NetCDFFile(fout,'a')
nclu_obs.variables['agc14box'][:] = agc14_new
nclu_obs.variables['bgc14box'][:] = bgc14_new
nclu_obs.close()

In [ ]:
#======== Following codes are for validation ==========#

In [ ]:
# 1. Read back in the initial file and calculate D14C for both two cases
fnew = 'bgc.isam_initial.adjust_continue.nc'
nclu_obs=NetCDFFile(fnew,'r')
agcbox = nclu_obs.variables['agcbox'][:]
bgcbox = nclu_obs.variables['bgcbox'][:]
agc14box = nclu_obs.variables['agc14box'][:]
bgc14box = nclu_obs.variables['bgc14box'][:]
latnc = nclu_obs.variables['lat'][:]
lonnc = nclu_obs.variables['lon'][:]
nclu_obs.close()

# 2. Read in PFT fraction data
flc = 'verify_time_invarient.nc'
nclu_obs=NetCDFFile(flc,'r')
lc = nclu_obs.variables['c_pft_fraction'][:]
nclu_obs.close()

# 3. Retreive back the gridded average C and C14 content profile
c14_box = agc14box.sum(axis=1)+bgc14box.sum(axis=1)
c_box = agcbox.sum(axis=1)+bgcbox.sum(axis=1)
c14_aggre_box = np.sum((c14_box * lc), axis=0)
c_aggre_box = np.sum((c_box * lc), axis=0)
    
# 4. Calculate the D14C profile
as_c14_std = 1e-12
# Add 1e-14 just to prevent divide by zero error
soc_as_c14 = (c14_aggre_box) / as_c14_std / (c_aggre_box+1e-14)
soc_del_c14 = (soc_as_c14 - 1.) * 1000.
    
# 5. Create a NetCDF file to store the result
nc = NetCDFFile('d14c_0d.nc','w',format='NETCDF4_CLASSIC')
# Define a set of dimensions to be used in the file
lat = nc.createDimension('lat', 360)
lon = nc.createDimension('lon', 720)
# Create variables
latitudes = nc.createVariable('lat', np.float32, ('lat',))
longitudes = nc.createVariable('lon', np.float32, ('lon',))
# Create the actual 4-d variable
d14c = nc.createVariable('d14c', np.float32, ('lat','lon'))

# Add Global Attributes
nc.description = 'D14C calculated from the 1D-SBGC restart file.'
nc.history = 'Created ' + time.ctime(time.time())
nc.source = 'netCDF4 python module'

# Add Variable Attributes
latitudes.units = 'degree_north'
longitudes.units = 'degree_east'
d14c.units = 'o/oo'

# Put longitude and latitude
lats = np.arange(-89.75,90.25,0.5)
lons = np.arange(0.25,360.25,0.5)
levels = np.arange(1,11)
latitudes[:] = lats
longitudes[:] = lons
d14c[:,:] = soc_del_c14
nc.close()

In [ ]:
fdv = 'dave_mask.nc'
nclu_obs=NetCDFFile(fdv,'r')
dv_msk = nclu_obs.variables['Davemask'][:]
nclu_obs.close()

In [ ]:
# Here read in the results and calculate the D14C for each layer
fsample = 'Global_1DSBGC.bgc-yearly-d14c-4d_1901.nc'
nclu_obs=NetCDFFile(fsample,'r')
d14c = nclu_obs.variables['soc_del_c14'][:]
nclu_obs.close()

# SOC profile
fsoc = 'Global_1DSBGC.bgc-yearly-3d_1901.nc'
nclu_obs=NetCDFFile(fsoc,'r')
soc = nclu_obs.variables['soil_c_lev'][:]
nclu_obs.close()

# LC data
flc = 'verify_time_invarient.nc'
nclu_obs=NetCDFFile(flc,'r')
lc = nclu_obs.variables['c_pft_fraction'][:]
nclu_obs.close()

# Need to get the mask to prevent running for useless grids.
fdv = 'dave_mask.nc'
nclu_obs=NetCDFFile(fdv,'r')
dv_msk = nclu_obs.variables['Davemask'][:]
nclu_obs.close()

d14c_all = -1000*np.ones((10, 360, 720))
for i in np.arange(0,10):
    d14c_all[i,:,:] = np.sum((d14c[i,:,:,:] * lc), axis=0)
    #d14c_all[i,:,:] = soc_del_c14[i,:,:]

d14c_all[d14c_all==0] = np.float('nan')
# Get a mask to filter out grids with no values
msk = d14c_all[0,:,:]
msk[dv_msk>0] = -1000
    
tau_topsoil = np.zeros((360, 720))
tau_subsoil = np.zeros((360, 720))
bulk_d14c_topsoil = -1000*np.ones((360, 720))
bulk_d14c_subsoil = -1000*np.ones((360, 720))

# Calculate the weighted-average mean for topsoil and subsoil
# We need to filter out the D14C above -50 for subsoil since these are contaminated grids.
# No need for current calculation now...
# for i in np.arange(0,720):
#     for j in np.arange(0,360):
#         for k in np.arange(5,8):
#             if(d14c_all[k,j,i]>-50):
#                 d14c_all[k,j,i] = np.float('nan')

for i in np.arange(0,720):
    for j in np.arange(0,360):
        if(soc[0,j,i]>0):
            bulk_d14c_topsoil[j,i] = isam.avg_wt_topsoil(soc[:,j,i], d14c_all[:,j,i])
            bulk_d14c_subsoil[j,i] = isam.avg_wt_subsoil(soc[:,j,i], d14c_all[:,j,i])

print 'Done the averaging!'

# Calculate the tau, one time for a lat
# Lat starts from 360 to 220 (20N ~ 80N), Dave Mcguire's regional mask is applied
# First transfer into a 1-D array then pass into cal_tau?
for i in np.arange(359,219,-1):
    print i
    # Weighted C14 for topsoil and subsoil separatedly
    # Preindustrial year - 1860
    # Contemporary year - 2000
    msk_inuse = msk[i,:]
    sampleyr = 1860*np.ones((720))
    sampleyr[msk_inuse<-900] = np.float('nan')
    # Calculate the turnover time
    tau, cost = C14utils.cal_tau(bulk_d14c_topsoil[i,:], sampleyr, 1, 0)
    tau_topsoil[i,:] = tau[:,0]
    tau, cost = C14utils.cal_tau(bulk_d14c_subsoil[i,:], sampleyr, 1, 0)
    tau_subsoil[i,:] = tau[:,0]

# # 7. Read in PFT fraction data
# flc = 'verify_time_invarient.nc'
# nclu_obs=NetCDFFile(flc,'r')
# lc = nclu_obs.variables['c_pft_fraction'][:]
# nclu_obs.close()

# # 8. Retreive back the gridded average C and C14 content profile
# d14c_aggre_prof=np.zeros((10,360,720))
# c_aggre_prof=np.zeros((10,360,720))
# for i in np.arange(0,10):
#     d14c_aggre_prof[i,:,:] = np.sum((d14c[i,:,:,:] * lc), axis=0)
#     c_aggre_prof[i,:,:] = np.sum((c_prof[i,:,:,:] * lc), axis=0)
    
# # 9. Calculate the D14C profile
# as_c14_std = 1e-12
# soc_as_c14 = np.zeros((10,360,720))
# soc_del_c14 = np.zeros((10,360,720))
# for i in np.arange(0,10):
#     # Add 1e-14 just to prevent divide by zero error
#     soc_as_c14[i,:,:] = (c14_aggre_prof[i,:,:]*1e-12) / as_c14_std / (c_aggre_prof[i,:,:]+1e-14)
#     soc_del_c14[i,:,:] = (soc_as_c14[i,:,:] - 1.) * 1000.
    
# # 10. Create a NetCDF file to store the result
# nc = NetCDFFile('d14c_out.nc','w',format='NETCDF4_CLASSIC')
# # Define a set of dimensions to be used in the file
# level = nc.createDimension('level', 10)
# lat = nc.createDimension('lat', 360)
# lon = nc.createDimension('lon', 720)
# # Create variables
# levels = nc.createVariable('level', np.int32, ('level',))
# latitudes = nc.createVariable('lat', np.float32, ('lat',))
# longitudes = nc.createVariable('lon', np.float32, ('lon',))
# # Create the actual 4-d variable
# d14c = nc.createVariable('d14c', np.float32, ('level','lat','lon'))

# # Add Global Attributes
# nc.description = 'D14C calculated from the 1D-SBGC restart file.'
# nc.history = 'Created ' + time.ctime(time.time())
# nc.source = 'netCDF4 python module'

# # Add Variable Attributes
# latitudes.units = 'degree_north'
# longitudes.units = 'degree_east'
# d14c.units = 'o/oo'

# # Put longitude and latitude
# lats = np.arange(-89.75,90.25,0.5)
# lons = np.arange(0.25,360.25,0.5)
# levels = np.arange(1,11)
# latitudes[:] = lats
# longitudes[:] = lons
# d14c[:,:,:] = soc_del_c14
# nc.close()

In [ ]:
# Save the file as netcdf for display
# Create a NetCDF file to store the result
nc = NetCDFFile('tau_out.nc','w',format='NETCDF4_CLASSIC')
# Define a set of dimensions to be used in the file
#level = nc.createDimension('level', 10)
lat = nc.createDimension('lat', 360)
lon = nc.createDimension('lon', 720)
# Create variables
#levels = nc.createVariable('level', np.int32, ('level',))
latitudes = nc.createVariable('lat', np.float32, ('lat',))
longitudes = nc.createVariable('lon', np.float32, ('lon',))
# Create the actual 4-d variable
tau_top = nc.createVariable('tau_top', np.float32, ('lat','lon'))
tau_sub = nc.createVariable('tau_sub', np.float32, ('lat','lon'))

# Add Global Attributes
nc.description = 'Tau calculated from the Soil D14C.'
nc.history = 'Created ' + time.ctime(time.time())
nc.source = 'netCDF4 python module'

# Add Variable Attributes
latitudes.units = 'degree_north'
longitudes.units = 'degree_east'
tau_top.units = 'year'
tau_sub.units = 'year'

# Put longitude and latitude
lats = np.arange(-89.75,90.25,0.5)
lons = np.arange(0.25,360.25,0.5)
latitudes[:] = lats
longitudes[:] = lons
tau_top[:,:] = tau_topsoil
tau_sub[:,:] = tau_subsoil
nc.close()

In [ ]:
## OLD METHOD seems not working... I now changed to use the calculated D14C for calculating back the soc14 based on soc of the new initial file
# 2. Obtain the C and 14C content 
# Open SBGC restart

#path='/data/jain1/c/sshu3/SBGC/data/NCSCDv2_Circumpolar_netCDF_05deg/ncscd_masked_0.5x0.5.nc'
nclu_obs=NetCDFFile(fc14,'r')
agcbox = nclu_obs.variables['agcbox'][:]
bgcbox = nclu_obs.variables['bgcbox'][:]
agcprof = nclu_obs.variables['agcprof'][:]
bgcprof = nclu_obs.variables['bgcprof'][:]
agc14box = nclu_obs.variables['agc14box'][:]
bgc14box = nclu_obs.variables['bgc14box'][:]
agc14prof = nclu_obs.variables['agc14prof'][:]
bgc14prof = nclu_obs.variables['bgc14prof'][:]
latnc = nclu_obs.variables['lat'][:]
lonnc = nclu_obs.variables['lon'][:]
nclu_obs.close()

nclu_obs=NetCDFFile(fbgc,'r')
agcbox_new = nclu_obs.variables['agcbox'][:]
bgcbox_new = nclu_obs.variables['bgcbox'][:]
agcprof_new = nclu_obs.variables['agcprof'][:]
bgcprof_new = nclu_obs.variables['bgcprof'][:]
agc14box_new = nclu_obs.variables['agc14box'][:]
bgc14box_new = nclu_obs.variables['bgc14box'][:]
agc14prof_new = nclu_obs.variables['agc14prof'][:]
bgc14prof_new = nclu_obs.variables['bgc14prof'][:]
nclu_obs.close()

soc14_agc = np.zeros((10,24,4,360,720))
soc14_bgc = np.zeros((10,24,5,360,720))
soc_agc = np.zeros((10,24,4,360,720))
soc_bgc = np.zeros((10,24,5,360,720))
soc14_agc_new = np.zeros((10,24,4,360,720))
soc14_bgc_new = np.zeros((10,24,5,360,720))
soc_agc_new = np.zeros((10,24,4,360,720))
soc_bgc_new = np.zeros((10,24,5,360,720))

# 3. Calculate back soc and soc14 profiles
for lev in np.arange(0,10):
    soc14_agc[lev,:,:,:,:] = agc14box * agc14prof[lev,:,:,:,:]/10000.
    soc14_bgc[lev,:,:,:,:] = bgc14box * bgc14prof[lev,:,:,:,:]/10000.
    soc_agc[lev,:,:,:,:] = agcbox * agcprof[lev,:,:,:,:]/10000.
    soc_bgc[lev,:,:,:,:] = bgcbox * bgcprof[lev,:,:,:,:]/10000.

c14c12_agc = soc14_agc/soc_agc
c14c12_bgc = soc14_bgc/soc_bgc

# Change the soc14 profiles based on the c14/c12 and soc profiles
for lev in np.arange(0,10):
    soc_agc_new[lev,:,:,:,:] = agcbox_new * agcprof[lev,:,:,:,:]/10000.
    soc_bgc_new[lev,:,:,:,:] = bgcbox_new * bgcprof[lev,:,:,:,:]/10000.

soc14_agc_new = c14c12_agc * soc_agc_new
soc14_bgc_new = c14c12_bgc * soc_bgc_new

# 4. Update the SOC14 pool size after getting the new profile
soc14_agc_tot = np.nansum(soc14_agc_new, axis=0)
soc14_bgc_tot = np.nansum(soc14_bgc_new, axis=0)

# Calculate the new profile factor
soc14_agcprof_new = np.zeros((10,24,4,360,720))
soc14_bgcprof_new = np.zeros((10,24,5,360,720))
for lev in np.arange(0,10):
    soc14_agcprof_new[lev,:,:,:,:] = soc14_agc_new[lev,:,:,:,:]/soc14_agc_tot * 10000.
    soc14_bgcprof_new[lev,:,:,:,:] = soc14_bgc_new[lev,:,:,:,:]/soc14_bgc_tot * 10000.
soc14_agcprof_new[soc14_agcprof_new<0]=0
soc14_bgcprof_new[soc14_bgcprof_new<0]=0
#soc14_agcprof_new.astype('int')
#soc14_bgcprof_new.astype('int')

# 5. Store back the SOC14 into the restart file
fout = 'bgc.isam_initial.1dbgc_new.nc'
nclu_obs=NetCDFFile(fout,'a')
nclu_obs.variables['agc14prof'][:] = soc14_agcprof_new.astype('int')
nclu_obs.variables['bgc14prof'][:] = soc14_bgcprof_new.astype('int')
nclu_obs.variables['agc14box'][:] = soc14_agc_tot
nclu_obs.variables['bgc14box'][:] = soc14_bgc_tot
nclu_obs.close()

In [ ]:
# Make Plot
# Can continue later
# Comparison of 1m SOC stock between two models and to the NCSCD data
fig = plt.figure(figsize=(30,24))

ax1 = fig.add_subplot(231)
ax1.set_title("ISAM-1D SOC stock (1m)",fontsize=32,pad=15)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
lon,lat = np.meshgrid(lonnc,latnc) #Returns coordinate matrices from coordinate vectors
x,y = map(lon,lat)
xs, ys = np.meshgrid(lonnc3, latnc3)
clevs = np.power(10.0, np.arange(-1.0,2.5,0.5))
cs1 = map.pcolormesh(x,y,soc_contemp_perm[:,:],
              norm=colors.Normalize(vmin=0,vmax=30.),cmap=plt.cm.YlOrBr) # use log color scale,
# Overlap scatter plot to show the peatland distribution
#map.scatter(xs, ys, s = 4*histosol, latlon=True, marker='_', color='cyan')
#map.scatter(xs, ys, s = 4*histel, latlon=True, marker='_', color='cyan')
cbar = map.colorbar(cs1,location='bottom',pad="5%",extend='both')
plt.annotate('(a)', fontsize=32, xy=(0.1, 0.9), xycoords='axes fraction')
#parallels = np.arange(45.,80.,15.)
## labels = [left,right,top,bottom]
#map.drawparallels(parallels)#,labels=[False,True,True,False])
#meridians = np.arange(15.,346.,30.)
#map.drawmeridians(meridians)#,labels=[True,False,False,True])

map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
cbar.ax.tick_params(labelsize=30) 

ax2 = fig.add_subplot(232)
ax2.set_title("ISAM-0D SOC stock (1m)",fontsize=32,pad=15)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs = map.pcolormesh(x,y,soc0d_contemp_perm[:,:],
              norm=colors.Normalize(vmin=0.,vmax=30.),cmap=plt.cm.YlOrBr) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
#map.scatter(xs, ys, s = 0.4*histosol, latlon=True, marker='_', color='cyan')
#map.scatter(xs, ys, s = 0.4*histel, latlon=True, marker='_', color='cyan')
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
cbar.ax.set_xlabel('Unit: $kgC m^{-2}$', fontsize=32, labelpad=10)
plt.annotate('(b)', fontsize=32, xy=(0.1, 0.9), xycoords='axes fraction')
cbar.ax.tick_params(labelsize=30) 

lon2,lat2 = np.meshgrid(lonnc2,latnc2) #Returns coordinate matrices from coordinate vectors
x2,y2 = map(lon2,lat2)
ax2 = fig.add_subplot(233)
ax2.set_title("NCSCD SOC stock (1m)",fontsize=32,pad=15)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs = map.pcolormesh(x2,y2,soc_ncscd_perm[:,:],
              norm=colors.Normalize(vmin=0.,vmax=30.),cmap=plt.cm.YlOrBr) # use log color scale,
                                                     # omit this to use linear
                                                     # color scale
#map.scatter(xs, ys, s = 0.4*histosol, latlon=True, marker='_', color='cyan')
#map.scatter(xs, ys, s = 0.4*histel, latlon=True, marker='_', color='cyan')
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
plt.annotate('(c)', fontsize=32, xy=(0.1, 0.9), xycoords='axes fraction')
cbar.ax.tick_params(labelsize=30) 

ax3 = fig.add_subplot(234)
ax3.set_title("ISAM-1D SOC turnover time",fontsize=32,pad=15)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs = map.pcolormesh(x,y,tau_contemp_perm[:,:],
              norm=colors.LogNorm(vmin=1,vmax=1500.),cmap=plt.cm.summer) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
cbar = map.colorbar(cs,ticks=[1, 10, 100, 1000, 5000],location='bottom',pad="5%",extend='both', format=ticker.LogFormatter())
plt.annotate('(a)', fontsize=32, xy=(0.1, 0.9), xycoords='axes fraction')
#cbar.ax.set_xticklabels([0, 200, 400, 600, 800])  # horizontal colorbar
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
cbar.ax.tick_params(labelsize=30) 

ax3 = fig.add_subplot(235)
ax3.set_title("ISAM-0D SOC turnover time",fontsize=32,pad=15)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs = map.pcolormesh(x,y,tau0d_contemp_perm[:,:],
              norm=colors.LogNorm(vmin=1,vmax=1500.),cmap=plt.cm.summer) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
cbar = map.colorbar(cs,ticks=[1, 10, 100, 1000, 5000],location='bottom',pad="5%",extend='both', format=ticker.LogFormatter())
cbar.ax.set_xlabel('Unit: year', fontsize=32, labelpad=10)
plt.annotate('(b)', fontsize=32, xy=(0.1, 0.9), xycoords='axes fraction')
#cbar.ax.set_xticklabels([0, 200, 400, 600, 800])  # horizontal colorbar
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
cbar.ax.tick_params(labelsize=30) 

#plt.show()
plt.savefig('./figures/fig6.png',dpi=300)

In [ ]:
# Mask all regions outside the northern permafrost extent
dsoc_perm = copy.deepcopy(dsoc)
dsoc_perm.mask = another_mask.mask
dsoc0d_perm = copy.deepcopy(dsoc0d)
dsoc0d_perm.mask = another_mask.mask
dtau_perm = copy.deepcopy(dtau)
dtau_perm.mask = another_mask.mask
dtau0d_perm = copy.deepcopy(dtau0d)
dtau0d_perm.mask = another_mask.mask

# Plot the figure7
# Comparison of 1m SOC stock between two models and to the NCSCD data
fig = plt.figure(figsize=(18,15))

#ax1 = fig.add_subplot(331)
ax1 = fig.add_subplot(221)
ax1.set_title("ISAM-1D SOC stock (2009 - 1960)",fontsize=20,pad=10)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
lon,lat = np.meshgrid(lonnc,latnc) #Returns coordinate matrices from coordinate vectors
x,y = map(lon,lat)
cs1 = map.pcolormesh(x,y,dsoc_perm[:,:],
              norm=colors.Normalize(vmin=-2.,vmax=2.),cmap=plt.cm.coolwarm) # use log color scale,
cbar = map.colorbar(cs1,ticks=[-2, -1, 0, 1, 2],location='bottom',pad="5%",extend='both')
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
plt.annotate('(a)', fontsize=24, xy=(0.1, 0.9), xycoords='axes fraction')
cbar.ax.tick_params(labelsize=22) 

ax2 = fig.add_subplot(222)
ax2.set_title("ISAM-0D SOC stock (2009 - 1960)",fontsize=20,pad=10)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs2 = map.pcolormesh(x,y,dsoc0d_perm[:,:],
              norm=colors.Normalize(vmin=-2.,vmax=2.),cmap=plt.cm.coolwarm) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
cbar = map.colorbar(cs2,ticks=[-2, -1, 0, 1, 2],location='bottom',pad="5%",extend='both')
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
cbar.ax.set_xlabel('Unit: $kgC m^{-2}$', fontsize=20, labelpad=0)
plt.annotate('(b)', fontsize=24, xy=(0.1, 0.9), xycoords='axes fraction')
cbar.ax.tick_params(labelsize=22) 


ax3 = fig.add_subplot(223)
ax3.set_title("ISAM-1D SOC turnover time (2009 - 1960)",fontsize=20,pad=10)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs3 = map.pcolormesh(x,y,dtau_perm[:,:],
#              norm=colors.Normalize(vmin=-300,vmax=300.),cmap=plt.cm.coolwarm) # use log color scale,
              norm=colors.SymLogNorm(linthresh=1, linscale=1, vmin=-300, vmax=300),cmap=plt.cm.coolwarm) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
cbar = map.colorbar(cs3,ticks=[-100, -10, -1, 0, 1, 10, 100],location='bottom',pad="5%",extend='both', format=ticker.LogFormatter())
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
plt.annotate('(c)', fontsize=24, xy=(0.1, 0.9), xycoords='axes fraction')
cbar.ax.tick_params(labelsize=22) 

ax4 = fig.add_subplot(224)
ax4.set_title("ISAM-0D SOC turnover time (2009 - 1960)",fontsize=20,pad=10)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs4 = map.pcolormesh(x,y,dtau0d_perm[:,:],
#              norm=colors.Normalize(vmin=-60.,vmax=60.),cmap=plt.cm.coolwarm) # use log color scale,
              norm=colors.SymLogNorm(linthresh=1, linscale=1, vmin=-300, vmax=300),cmap=plt.cm.coolwarm) # use log color scale,

cbar = map.colorbar(cs4,ticks=[-100, -10, -1, 0, 1, 10, 100],location='bottom',pad="5%",extend='both', format=ticker.LogFormatter())
cbar.ax.set_xlabel('Unit: $year$', fontsize=20, labelpad=0)
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
plt.annotate('(d)', fontsize=24, xy=(0.1, 0.9), xycoords='axes fraction')
cbar.ax.tick_params(labelsize=22) 

#plt.show()
plt.savefig('./figures/fig7.png',dpi=300)

In [ ]:
dsoc_top_perm = soc_top_contemp_perm - soc_top_preind_perm
dsoc_sub_perm = soc_sub_contemp_perm - soc_sub_preind_perm

fig = plt.figure(figsize=(18,10))

ax1 = fig.add_subplot(121)
#ax1.set_title("ISAM 1D topsoil SOC change (kgC/m2)",fontsize=20,pad=10)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
lon,lat = np.meshgrid(lonnc,latnc) #Returns coordinate matrices from coordinate vectors
x,y = map(lon,lat)
clevs = np.power(10.0, np.arange(-1.0,2.5,0.5))
cs1 = map.pcolormesh(x,y,dsoc_top_perm[:,:],
              norm=colors.Normalize(vmin=-2.,vmax=2.),cmap=plt.cm.coolwarm) # use log color scale,
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
cbar = map.colorbar(cs1,ticks=[-2, -1, 0, 1, 2],location='bottom',pad="5%",extend='both')
cbar.ax.tick_params(labelsize=22) 
plt.annotate('(a)', fontsize=24, xy=(0.1, 0.9), xycoords='axes fraction')

ax2 = fig.add_subplot(122)
#ax2.set_title("ISAM 1D subsoil SOC change (gC/m2/year)",fontsize=20,pad=10)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs = map.pcolormesh(x,y,dsoc_sub_perm[:,:],
              norm=colors.Normalize(vmin=-2.,vmax=2.),cmap=plt.cm.coolwarm) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
cbar = map.colorbar(cs,ticks=[-2, -1, 0, 1, 2],location='bottom',pad="5%",extend='both')
cbar.ax.tick_params(labelsize=22) 
cbar.ax.set_xlabel('Unit: $kgC m^{-2}$', fontsize=20, labelpad=10)
plt.annotate('(b)', fontsize=24, xy=(0.1, 0.9), xycoords='axes fraction')

# Set one colorbar for both subplots
# make an axes to put the shared colorbar in
#fig.subplots_adjust(bottom=0.1)
#colorbar_axes = plt.gcf().add_axes([0.35, 0.1, 0.3, 0.05])
#colorbar = plt.colorbar(cs, colorbar_axes, orientation='horizontal')
#colorbar_axes.tick_params(labelsize=20) 
#colorbar.set_label('Unit: $KgC m^{-2}$', fontsize=24)

#plt.show()
plt.savefig('./figures/fig8.png',dpi=300)

In [ ]:
# Plot the figure3
# Mask all regions outside the northern permafrost extent
resp_contemp_perm = copy.deepcopy(resp_contemp)
resp_contemp_perm.mask = another_mask.mask
resp0d_contemp_perm = copy.deepcopy(resp0d_contemp)
resp0d_contemp_perm.mask = another_mask.mask
resp_preind_perm = copy.deepcopy(resp_preind)
resp_preind_perm.mask = another_mask.mask
resp0d_preind_perm = copy.deepcopy(resp0d_preind)
resp0d_preind_perm.mask = another_mask.mask
dresp_perm = copy.deepcopy(dresp)
dresp_perm.mask = another_mask.mask
dresp0d_perm = copy.deepcopy(dresp0d)
dresp0d_perm.mask = another_mask.mask

# Comparison of Soil respiration and the change
fig = plt.figure(figsize=(18,15))

ax1 = fig.add_subplot(221)
ax1.set_title("ISAM 0D Heterotrophic respiration (gC/m2/year)",fontsize=20,pad=10)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
lon,lat = np.meshgrid(lonnc,latnc) #Returns coordinate matrices from coordinate vectors
x,y = map(lon,lat)
clevs = np.power(10.0, np.arange(-1.0,2.5,0.5))
cs1 = map.pcolormesh(x,y,1000.*resp0d_contemp_perm[:,:],
              norm=colors.Normalize(vmin=0.,vmax=500.),cmap=plt.cm.jet) # use log color scale,
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
cbar = map.colorbar(cs1,location='bottom',pad="5%",extend='both')
cbar.ax.tick_params(labelsize=22) 

ax2 = fig.add_subplot(222)
ax2.set_title("ISAM 0D change of Rh (gC/m2/year)",fontsize=20,pad=10)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs = map.pcolormesh(x,y,1000.*dresp0d_perm[:,:],
              norm=colors.Normalize(vmin=-50.,vmax=80.),cmap=plt.cm.RdYlGn_r) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
cbar.ax.tick_params(labelsize=22) 

ax3 = fig.add_subplot(223)
ax3.set_title("ISAM 1D Heterotrophic respiration (gC/m2/year)",fontsize=20,pad=10)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs = map.pcolormesh(x,y,1000.*resp_contemp_perm[:,:],
              norm=colors.Normalize(vmin=0.,vmax=500.),cmap=plt.cm.jet) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
cbar.ax.tick_params(labelsize=22) 

ax4 = fig.add_subplot(224)
ax4.set_title("ISAM 1D change of Rh (gC/m2/year)",fontsize=20,pad=10)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs = map.pcolormesh(x,y,1000.*dresp_perm[:,:],
              norm=colors.Normalize(vmin=-50.,vmax=80.),cmap=plt.cm.RdYlGn_r) # use log color scale,
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
cbar.ax.tick_params(labelsize=22) 

plt.show()
#plt.savefig('Fig9.jpg',dpi=300)